# Assignment ML

## Setup for assignment

In [30]:
from random import Random
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics

In [31]:
df = pd.read_csv(r"C:\Users\Vincent Lau\Google Drive\Quantitative finance\Machine learning\ml_assignment\data\train.csv",delimiter=";", decimal=",")


In [ ]:
from cgi import test


train_df = df.sample(frac=0.6)

train_df_x = train_df.iloc[:,1:]
train_df_y = train_df.iloc[:,0]

test_df = df.drop(train_df.index)

test_df_x = test_df.iloc[:,1:]
test_df_y = test_df.iloc[:,0]

## Early Stopping

In [33]:
## Early stopping code
early_stopping_value = 0
early_stopping_accuracy = 0
for i in np.linspace(0.1,0.9,100):
    train_df = df.sample(frac=i)
    train_df_x = train_df.iloc[:,1:]
    train_df_y = train_df.iloc[:,0]

    test_df = df.drop(train_df.index)

    test_df_x = test_df.iloc[:,1:]
    test_df_y = test_df.iloc[:,0]
    
    clf = RandomForestClassifier(n_estimators=100, criterion="gini", max_depth=None)
    clf.fit(train_df_x, train_df_y)


    #Predict
    clf.predict(test_df_x)
    
    
    if (clf.score(test_df_x,test_df_y) - early_stopping_accuracy > 0.1):
        early_stopping_value = i
        early_stopping_accuracy = clf.score(test_df_x,test_df_y)
    
print("Best fraction to stop:", early_stopping_value)    

Best fraction to stop: 0.5606060606060607


## Tuning parameters

### Grid search


In [34]:
train_df = df.sample(frac=0.5)

train_df_x = train_df.iloc[:,1:]
train_df_y = train_df.iloc[:,0]

n_trees = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 100)]
max_features = [int(x) for x in np.linspace(start = 5, stop = 70, num = 5)]
criterion = ['gini', 'entropy']
max_depth = [int(x) for x in np.linspace(start = 3, stop = 100, num = 10)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 100, num = 10)]

random_grid = {'n_estimators': n_trees,
               'max_features': max_features,
               'criterion': criterion,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split}

rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions= random_grid, n_iter = 100, cv=5)

rf_random.fit(train_df_x, train_df_y)

KeyboardInterrupt: 

In [ ]:
# Print the randomized grid search parameters
# Results: n_estimators = 881, min_samples_split = 2, max_features = 70, max_depth = 78, criterion = entropy
print(rf_random.best_params_)

{'n_estimators': 881, 'min_samples_split': 2, 'max_features': 70, 'max_depth': 78, 'criterion': 'entropy'}


### Check for overfitting using the randomized grid search

In [35]:
rf2 = RandomForestClassifier(n_estimators=881, min_samples_split=2, max_features=70, max_depth=60, criterion='entropy')
rf3 = RandomForestClassifier(n_estimators=881, min_samples_split=2, max_features=70, max_depth=60, criterion='entropy')

train_df = df.sample(frac=.6)
train_df_x = train_df.iloc[:,1:]
train_df_y = train_df.iloc[:,0]

test_df = df.drop(train_df.index)

test_df_x = test_df.iloc[:,1:]
test_df_y = test_df.iloc[:,0]


rf2.fit(train_df_x, train_df_y)
rf3.fit(test_df_x, test_df_y)

#Predict
rf2predict=rf2.predict(test_df_x)
rf3predict=rf3.predict(train_df_x)

print("rf2:",rf2.score(test_df_y,rf2predict))
print("rf3:", rf3.score(train_df_y,rf3predict))


ValueError: Expected 2D array, got 1D array instead:
array=[0. 1. 0. ... 0. 0. 0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

## Another randomized grid search using the values from the previous run

In [36]:
train_df = df.sample(frac=0.5)

train_df_x = train_df.iloc[:,1:]
train_df_y = train_df.iloc[:,0]

n_trees = [int(x) for x in np.linspace(start = 750, stop = 1500, num = 100)]
max_features = [int(x) for x in np.linspace(start = 5, stop = 70, num = 5)]
criterion = ['gini', 'entropy']
max_depth = [int(x) for x in np.linspace(start = 10, stop = 80, num = 20)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 20, num = 10)]

random_grid = {'n_estimators': n_trees,
               'max_features': max_features,
               'criterion': criterion,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split}

rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions= random_grid, n_iter = 100, cv=5)

rf_random.fit(train_df_x, train_df_y)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=100,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [10, 13, 17, 21, 24, 28,
                                                      32, 35, 39, 43, 46, 50,
                                                      54, 57, 61, 65, 68, 72,
                                                      76, 80],
                                        'max_features': [5, 21, 37, 53, 70],
                                        'min_samples_split': [2, 4, 6, 8, 10,
                                                              12, 14, 16, 18,
                                                              20],
                                        'n_estimators': [750, 757, 765, 772,
                                                         780, 787, 795, 803,
                                                         810, 818, 825, 833,
                         

In [37]:
print('Random grid search params:', rf_random.best_params_)
#Random grid search params: {'n_estimators': 856, 'min_samples_split': 4, 'max_features': 70, 'max_depth': 24, 'criterion': 'entropy'}

Random grid search params: {'n_estimators': 856, 'min_samples_split': 4, 'max_features': 70, 'max_depth': 24, 'criterion': 'entropy'}


In [40]:
rf3 = RandomForestClassifier(n_estimators=856, min_samples_split=4, max_features=70, max_depth=24, criterion='entropy')

train_df = df.sample(frac=.6)
train_df_x = train_df.iloc[:,1:]
train_df_y = train_df.iloc[:,0]

test_df = df.drop(train_df.index)

test_df_x = test_df.iloc[:,1:]
test_df_y = test_df.iloc[:,0]


rf3.fit(train_df_x, train_df_y)

#Predict
rf3predict=rf3.predict(test_df_x)

print("rf3:", rf3.score(test_df_x,test_df_y))

rf3: 0.912630890052356
